# 가설과 추론

지금까지 배운 통계와 확률 이론을 활용하여
가설을 세우고 검정하는 방법을 배운다.

여기서 다루는 주제는 다음과 같다.

1. 통계적 가설검정
1. $p$-값
1. 신뢰구간
1. A/B 테스트
1. 베이지안 추론

## 통계적 가설 검정

통계적 가설 검정은 
모집단에 대한 가설을 세운 후 표본을 통해 얻은 통계를 이용하여 
해당 가설을 검정(test)하는 것을 의미한다.

**참고**: 검정(test), 테스팅(testing), 검증(verification)은 조금씩 다른 의미로 사용된다. 

* 검정: 일정 기준을 만족하는지 여부 판단
* 테스팅: 특정 기능을 잘 수행하는지 여부 판단
* 검증: 특정 성질을 만족하는지 여부 판단

### 귀무가설과 대립가설

검정 대상인 가설을 **귀무가설**($H_0$, null hypothesis), 
그에 대립되는 가설을 **대립가설**($H_1$, alternative hypothesis)라 부른다. 

통계를 사용하여 귀무가설 $H_0$를 받아들일지, 아니면 기각하고 대립가설 $H_1$을 받아들일지를 결정한다.
귀무가설을 검정할 때 발생할 수 있는 오류에는 두 종류가 있다. 

* **제1종 오류**: 귀무가설 $H_0$가 참이지만 기각하는 오류
* **제2종 오류**: 귀무가설 $H_0$가 거짓이지만 받아들이는 오류

그리고 제2종 오류가 발생하지 않을 확률을 **검정력**이라 부른다.

## 예제: 동전 던지기

동전 던지기를 이용하여 가설검정 방법을 설명한다. 

지금까지는 동전이 정상적인 동전인지 아니면 앞면 또는 뒷면이 보다 많이 나오는 편향된 동전인지 
알고서 특정 사건이 발생할 확률을 구했다. 
그런데 이제는 동전에 대한 정보를 모른다고 가정한다.
그리고 나서 모의실험 결과를 확인한 후에 동전에 대한 정보를 통계적으로 추정한다.
이게 바로 **통계적 가설 검정**이라고 부르는 이유이다. 

동전을 $n$번 던져서 앞면이 나오는 횟수를 가리키는 확률 변수 $X$에 대한 가설을 검정해보자.
동전을 한 번 던져서 앞면이 나올 확률을 $p$라 할 때,
$X$는 베르누이 확률 변수로 간주될 수 있으며, 
따라서 다음 모양의 이항 분포를 따른다. 

$$X \sim B(n, p)$$

또한 앞서 살펴 보았듯이 $n$이 충분히 크면 $X$는
평균값(mean)은 $\mu = n p$ 이고 
표준편차는 $\sigma = \sqrt{n p (1-p)}$ 인 
정규분포를 따른다.
즉, 다음이 성립한다.

$$X \sim N(n p, n p (1-p))$$


이제 동전을 1,000번 던지는 모의실험을 실행했다고 가정하자. 
즉, $n = 1000$ 이다.


그리고 검정해야할 귀무가설 $H_0$는 '동전이 정상이다'로 정한다.

* $H_0$: $p = 0.5$

$n = 1,000$이면 충분히 크므로 중심극한정리에 의해 다음이 성립한다. 
($1,000 \times 0.5 = 500 > 5$)

$$X \sim N(\mu_0, \sigma_0^2) = N(500, 250) = N(500, 15.8^2)$$

$\mu_0$와 $\sigma_0$ 계산은 매우 쉽다.

In [34]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """
    X ~ B(n,p)이고 n이 충분히 클 때,
    평균값 mu와 표준편차 sigma 계산 
    """
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [22]:
mu0, sigma0 = normal_approximation_to_binomial(1000, 0.5)
print(f"평균:\t{mu0}")
print(f"표준편차:\t{sigma0:.1f}")

평균:	500.0
표준편차:	15.8


### 정규분포 관련 함수 정리

정규분포를 이용한 가설검정을 수행하기 위해 필요한 관련 함수들을 먼저 확인하고자 한다. 
아래 코드는 이전에 사용한 코드를 모아 놓은 '../scratch' 폴더에 있는
파이썬 모듈을 불러오기 위해 필요다. 

In [23]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

가설검정은 발생한 특정 사건이 지정된 확률 분포 구간에 포함되는지 
여부를 확인하여 귀무가설을 받아들일지 말지를 판단한다.

여기서는 정규분포에서 확률 변수가 특정 값보다 작을 확률을 
계산하는 누적 분포 함수 `normal_cdf()` 를 이용하여, 
확률 변수가 특정 구간에 포함될 확률을 계산하는 도우미 함수 네 개를 정의한다. 

* `normal_probability_below`: 확률 변수가 특정 값보다 작을 확률
* `normal_probability_above`: 확률 변수가 특정 값보다 클 확률
* `normal_probability_between`: 확률 변수가 특정 구간에 위치할 확률
* `normal_probability_outside`:확률 변수가 특정 구간 밖에 위치할 확률

In [24]:
from scratch.probability import normal_cdf

normal_probability_below = normal_cdf

def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    return 1 - normal_probability_between(lo, hi, mu, sigma)

또한 누적 분포 함수의 역함수를 이용하여 확률이 주어졌을 때 평균값을 중심으로
좌우 어느 정도의 구간에 해당하는지를 계산하는 것과 관련된 함수 세 개를 정의한다.

* `normal_upper_bound(p)`: $P(Z \le z)=p$ 인 $z$ 계산
* `normal_lower_bound(p)`:$P(Z \ge z)=p$ 인 $z$ 계산
* `normal_two_sided_bound(p)`:$P(z_1 \le Z \le z_2)=p$ 인 $z_1, z_2$ 계산

In [25]:
from scratch.probability import inverse_normal_cdf

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    tail_probability = (1 - probability) / 2

    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

### 유의수준

귀무가설을 검정하려면 먼저 **유의수준**(significance)을 지정해야 한다.
유의수준은 보통 5% 또는 1%로 정하며, 여기서는 5%를 사용한다.

유의수준이 5%라는 것은 
귀무가설이 정말로 참이라고 해도 
발생확률이 5%가 안되는 사건이 모의실험에서 발생하면 
어쩔 수 없이 귀무가설을 기각해야 한다는 것을 의미한다.
즉, 제1종 오류를 범할 확률이 5%라는 의미이다.

### 양측검정과 단측검정

가설검정 방법에 크게 두 가지 방법이 있다.

* **양측검정**(two_sided test): 귀무가설이 '...와 같다'의 형식인 경우 사용
* **단측검정**(one-sided test)
    * 상단측검정: 귀무가설이 '...보다 작다'의 형식인 경우 사용
    * 하단측검정: 귀무가설이 '...보다 크다'의 형식인 경우 사용

### 기각역

유의수준에 따라 귀무가설을 기각하기 위한 기준으로 사용되는 값들의 집합을 **기각역**이라 하며,
가설검정 방식에 따라 위치와 크기가 달라진다.

아래 그래프는 차례대로 유의수준 5%에서 양측검정, 상단측검정, 하단측검정의 기각역을 보여준다.

<p>
<table cellspacing="20">
<tr>
    <td><img src="../images/two-sided-region.png"></td>
    <td><img src="../images/one-sided-region-up.png"></td>
    <td><img src="../images/one-sided-region-down.png"></td>
</tr>
<tr>
    <td>양측검정</td>
    <td>상단측검정</td>
    <td>하단측검정</td>
</tr>
</table>
</p>

### 동전 던지기와 양측검정 예제

귀무가설이 $H_0: p = 0.5$, 즉 '동전이 정상적이다'라는 가설을 검정해야 하므로,
대립가설을 '동전이 정상적이 아니다'에 해당하는 $H_1: p \neq 5$로 정한다.
그리고 이럴 경우 양측검정을 사용해서 가설검정을 실행한다.

유의수준 5%로 양측검정을 실행할 때, 기각역은 위 첫째 그래프에서 보이는 것처럼
양끝의 2.5%에 해당하는 구간이다. 
실제로 확인해보면 다음과 같다.

In [33]:
from math import ceil, floor

lower, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

print(f"하단기각역: {floor(lo)} 이하")
print(f"상단기각역: {ceil(hi)} 이상")

하단기각역: 469 이하
상단기각역: 531 이상


위 결과의 의미는 다음과 같다.

귀무가설이 참일 때, 즉, 정상적인 동전을 1,000번 던지면

* 2.5%의 확률로 469회 이하 앞면이 나오고,
* 2.5%의 확률로 531회 이상 앞면이 나온다.
* 즉, 앞면이 나오는 횟수가 469에서 531 사이에 발생할 확률이 95%이다.

**주의** 'k가 m에서 n 사이이다'의 의미는 'm < k < n'을 의미한다. 

<img src="../images/p-value_469531.png" width="50%">

따라서 모의실험 결과 앞면이 나온 횟수가 469회 이하 또는 531회 이상이면 
발생가능성이 5%로 매우 희박한 일이 발생하였다고 보고
귀무가설을 기각한다.

물론, 확률이 5%라는 게 매우 낮은 수치이기는 하지만 0%는 아니기에 정상적인 동전을 1,000번
던졌을 때 발생할 수는 있다.
이런 의미에서 옳은 귀무가설을 기각하는 잘못을 저지를 확률이 바로 유의수준 5%의 의미이다.

### 동전 던지기와 검정력 예제

검정력(test power)은 틀린 귀무가설을 기각하지 않는 확률인
**제2종 오류**를 범하지 않을 확률이다.
따라서 검정력을 계산하려면 제2종 오류를 범할 확률을 구해야 한다.

예를 들어, 동전 앞면이 나올 확률이 $p=0.55$로 앞면쪽으로 약간 편향되어 있다고 하자.
그러면 동전을 1,000번 던졌을 때 앞면이 95%의 확률로 
520회에서 580회 사이에 발생해야 한다.

In [36]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
lo_1, hi_1 = normal_two_sided_bounds(0.95, mu_1, sigma_1)

print(f"확률 95% 구간: ({floor(lo_1)}, {ceil(hi_1)})")

확률 95% 구간: (519, 581)


그런데 정상적인 동전을 1,000번 던져도 95%의 확률로 469회에서 531회 사이에 앞면이 발생한다. 
이런 의미에서 제2종 오류를 범할 확률은
$p=0.55$인 동전을 1,000번 던졌을 때 
469회에서 531회 사이에 앞면이 나올 확률을 계산하면 된다. 
계산된 확률은 11.3%이다.

아래 코드에서 `mu_1, sigma_1`이 사용되었음에 주의하라.
이는 편향된 동전을 기준으로 앞면이 469회에서 531회 사이로 나올 확률을 계산하는 것을 
의미한다. 그리고 그런 사건이 벌어지면 어쩔 수 없이 정상적인 동전이라고 가정하는 수밖에 없다.

In [40]:
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
print(f"{type_2_probability:.3f}")

0.113


따라서 검정력은 88.7%이다.

In [39]:
power = 1 - type_2_probability
print(f"{power:.3f}")

0.887


### 동전 던지기와 단측검정 예제

앞서 검정력을 양측검정을 이용하여 계산하였다.
하지만 귀무가설을 '동전이 앞면에 편향되지 않았다'로 정하면 상단측검정을 이용해야 한다.
즉, 귀무가설과 대립가설을 아래와 같이 설정한다.

* $H_0: p \le 0.5$
* $H_1: p > 0.5$

따라서 유의수준 5%의 기각역은 $p=0.5$일 때 상단 5%에 해당하는 구간이다.
이유는 $p\le 0.5$ 일 때, 95%의 확률로 나올 수 있는 앞면의 횟수의 최댓값은
$p=0.5$일 때 95%의 확률로 나올 수 있는 앞면의 횟수의 최댓값이기 때문이다. 

아래 코드에서 확인할 수 있듯이 기각역은 527부터 시작한다.
즉, 앞면이 526회 이하로 나오면 나오면 정상적인 동전을 던졌을 때 나올 확률이 5% 이하인 경우가 발생한 것이다.

In [52]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(f"{hi:.3f}")

526.007


<img src="../images/p-value_527.png" width="50%">

따라서 $p= 0.55$인 동전이라 하더라도 동전을 1,000번 던졌을 때 
526회 이하로 앞면이 나오면 귀무가설을 기각할 수 없다. 
따라서 아래 코드가 계산해주듯이 제2종 오류를 범할 확률은 6.4%이다.

In [51]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
print(f"{type_2_probability:.3f}")

0.064


즉, 검정력이 93.6%로 높아진다.

In [47]:
power = 1 - type_2_probability 
print(f"{power:.3f}")

0.936


### 연습문제

동전 던지기를 이용하여 하단측검정 문제를 설정하고 검정력을 계산하라.

## $p$-값

$p$-값은 가설검정을 다른 관점에서 진행할 때 계산하는 값이다. 

앞서 동전 던지기 예제에서 귀무가설을 가정하고, 동전을 1,000번 던졌을 때
앞면이 나오는 횟수가 95%의 확률로 '469에서 531 사이' 이고,
따라서 그 범위를 벗어나면 귀무가설을 기각한다 라고 하였다.
즉, 가설검정 과정을 요약하면 다음과 같다.

1. 귀무가설을 전제로 하는 확률 분포 설정
1. 지정된 유의수준을 이용하여 기각역 계산
1. 모의실험 결과가 기각역에 포함되는지 여부 판단
1. 귀무가설 기각 여부 판단

하지만 이 설명을 아래와 같이 다르게 설명할 수 있다.

1. 모의실험 결과를 확인한다.
    * 예를 들어, 1,000번 던져서 앞면이 530번 관측되었다고 하자.
    <br><br>

2. 귀무가설이 참일 때, 모의실험 결과가 나올 확률을 계산한다. 계산된 활률값을 **$p$-값**이라 부른다.
    * 예를 들어, 앞면이 530회 이상 발생할 확률 $p$ 를 계산한다.
    * 530회 이상 발생할 확률을 구하는 이유는 주사이의 편향 여부를 판단하기 위해서이다.
    <br><br>
3. $p$-값을 유의수준과 비교한다.
    * 양측검정의 경우
        * 유의수준의 **절반** 보다 크면, 모의실험 결과가 기각역 밖에 위치한다는 의미이며 따라서 귀무가설을 받아들인다.
        * 유의수준의 **절반** 보다 작으면, 모의실험 결과가 기각역 안에 위치한다는 의미이며 따라서 귀무가설을 기각한다.
    * 단측검정의 경우
        * 유의수준 보다 크면, 모의실험 결과가 기각역 밖에 위치한다는 의미이며 따라서 귀무가설을 받아들인다.
        * 유의수준 보다 작으면, 모의실험 결과가 기각역 안에 위치한다는 의미이며 따라서 귀무가설을 기각한다.

### 양측검정과 $p$-값

`two_sided_p_value()` 함수는 양측검정 기준으로 특정 사건이

?????
발생할 확률을 계산한다.
계산 결과가 유의수준 보다 크면 해당 사건이 기각역 밖에 존재한다는 의미이다. 

In [59]:
def two_sided_p_value(x: float, 
                      mu: float = 0, 
                      sigma: float = 1) -> float:
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

#### 예제

정상적인 동전을 던져서 앞면이 530번 이상 나올 확률은 6.2%이며, 5% 보다 크다.
이는 530이 기각역 밖에 존재한다는 의미이며, 따라서 귀무가설을 받아들인다.

In [58]:
# 530 대신에 529.5를 사용하는 이유는 연속성 보정 때문이다.
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

<img src="../images/p-value_530.png" width="50%">

### 연속성 보정

동전 던지기는 전형적인 이산 확률분포의 한 종류인 이항분포이다.
반복횟수가 크면 중심극한정리에 연속확률분포인 정규분포와 비슷한 확률분포가 되지만
그래도 실제로는 이산 확률분포이다.
따라서 정규분포를 이용해 확률을 계산할 때 오차를 보정해줄 필요가 있다.
이를 **연속성 보정**(continuity correction)이라 부른다.

연속성 보정을 하는 방법은 간단하다. 

* 어떤 사건이 $k$회 이상 나올 확률을 계산 할 때, 보통 $k-0.5$ 이상에 대한 확률을 계산한다.
* 어떤 사건이 $k$회 이하 나올 확률을 계산 할 때, 보통 $k+0.5$ 이하에 대한 확률을 계산한다.

이는 수학적으로 근거가 있다. 
확률이 확률밀도함수로 둘러싸인 면적을 적분으로 계산한다.
따라서 아래 그림에서 보이는 것처럼 보정을 해주는 것이 합리적이다.

아래 그림에서 보면 선그래프로 면적을 계산하는 것과 막대들의 면적으로 계산하는 것 사이에 오차가
있을 알 수 있다. 
또한 $\pm 0.5$를 해주는 이유 또한 그래프에서 확인할 수 있다.

<img src="../images/continuity_correction.png" width="50%">

### 모의실험으로 확인하기

정규분포 통계를 이용하지 않고, 모의실험을 통해 위 결과를 확인할 수 있다.
즉, 동전 1000번 던지기를 10,000번 반복하여 1,000번 던질 때 앞면이 530회 이상 나오는 확률을 
모의실험해 보면 거의 비슷한 결과를 얻는다.

In [10]:
import random

random.seed(1000)

extreme_value_count = 0
for _ in range(10000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range(1000))            
    if num_heads >= 530 or num_heads <= 470:         
        extreme_value_count += 1                     

extreme_value_count / 10000

0.0617

반면에 모의실험 결과 앞면이 532번 나오면
532번 이상 나오는 사건에 대한 $p$-값은 4.63%이며, 5% 이하이다.
따라서 귀무가설을 기각해야 한다. 

In [11]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

### 단측검정과 $p$-값

단측검정을 하는 경우 기각역이 달라진다.

예를 들어 동전이 525회 나왔다면 $p$-값은 6.1%이며, 따라서 기무가설을 기각하지 않는다.

반면에 527회 나오면, $p$-값은 4.7%이며, 따라서 기무가설을 기각한다. 

In [12]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

print("525회 이상에 대한 p-값:", upper_p_value(524.5, mu_0, sigma_0))
print("527회 이상에 대한 p-값:", upper_p_value(526.5, mu_0, sigma_0))

525회 이상에 대한 p-값: 0.06062885772582083
527회 이상에 대한 p-값: 0.04686839508859242


<img src="../images/p-value_525.png" width="50%">

### 주의사항

* 검정가설은 다루는 확률분포가 정규분포를 따른다는 전제하에 진행된다.
    정규분포가 아닐 경우 앞서 설명한 검정가설은 전혀 의미가 없다.

* 정규분포를 따르는가 여부를 확인하려면 앞서 설명한 방식으로 모의실험을 진행해서 그래프로 그려보면 된다.

## 신뢰구간과 신뢰수준

가설 검정을 할 때 어떤 사건이 발생할 확률 $p$를 정확히 안다고 가정했다.
예를 들어, 동전 던지기의 경우, 정상적인 동전이라면 앞면이 나올 확률은 $p = 0.5$이며,
이것과 모의실험 결과를 이용하여 가설 검정을 진행하였다.

반면에 대통령지지율 등과 같은 것은 정확히 알 수 없다.
따라서 사람들의 표본을 추출하여 설문조사를 진행하였는데 지지율이 49%로 나왔을 때,
그 결과를 정확히 검증할 수 있는 방법이 없다. 

얘를 들어, 2019년 12월 현재, 대한민국 인구수는 총 5,200만명 정도이며, 이중 20세 이상이 4,300만명 정도이다.
그런데 설문조사는 보통 1~2천명 정도를 대상으로 진행된다.
그리고 설문조사 결과가 49%의 지지율일 때, 그 결과를 어느 정도 신뢰할 수 있을까?
이런 질문에 통계적으로 대답하기 위해 신뢰구간을 사용한다. 

**신뢰구간**은 표본을 대상으로 값을 이용하여 
모집단에 대한 값이 속할 수 있는 구간을 통계적 지정한 구간을 의미한다.
이때 구간의 크기는 **신뢰수준**이 결정한다.

예를 들어, 대통령 지지율 설문조사 결과가 49%라고 해서 대통령 지지율이 49%입니다 라고
단정해서 공표하지 않는다.
신문이나 TV 뉴스 등에서 대통령 지지율을 공표할 때는 반드시 신뢰수준을 언급하는 데,
보통 95%를 사용한다. 

이는 대통령 지지율이 95%의 확률로 $49\pm \alpha$ 의 구간에 위치한다는 의미이다.
여기서 알파($\alpha$)는 보통 **오차범위**라고 불리기도 한다.
예를 들어, 대통령 선거에서 '두 후보가 오차범위 안에서 접전이다' 라는 표현이
많이 사용되며 여기서 언급된 오차범위가 바로 $\alpha$이다.

오차범위 $\alpha$ 의 값은 사용하는 확률분포와 표본의 크기에 의존하며,
자세한 설명은 여기서는 생략한다.
확률과 통계를 다루는 모든 서적에 포함되어 있는 내용이기도 하다.



<img src="../images/confidence-interval.png" width="50%">

### 동전 던지기와 신뢰구간

신뢰구간을 사용하여 가설 검정을 진행할 수 있다.

예를 들어, 동전을 1,000번 던졌더니 앞면이 525회 나왔다고 하자.
이때 동전이 앞면으로 편향되어 있다고 말할 수 있을까?
즉, 동전이 정상이다 라는 귀무가설을 기각할 수 있을까?

이제 신뢰수준 95%로 동전에 대한 신뢰구간을 알아보자.

일단 동전 실체에 대한 정보가 없기에, 모의실험 결과를 이용해야 한다.
즉, 앞면이 나올 확률을 $p = 525/1000 = 0.525$ 라고 한다.
그리고 이를 이용해 표준편차 $\sigma$를 계산하면 다음과 같다.

$$\sigma = \sqrt{\frac{p \cdot (1-p)}{1000}} = 0.0158$$

**주의:** 표본의 크기가 $n$일 때 표본들의 평균과 표준편차 계산은 다음과 같다.

* $\mu = p$
* $\sigma = \sqrt{\frac{p \cdot (1-p)}{n}}$

In [13]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)

0.015791611697353755


이제 $N(0.525, 0.0158^2)$을 따른 정규분포에서 평균을 중심으로 한 영역이 95%인 구간은 아래와 같다.

$$[0.4940, 0.5560]$$

In [14]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

정상적인 동전의 경우 앞면이 나올 확률이 0.5인데 위 구간에 포함된다.
따라서 앞면이 525회 나온 것 때문에 정상적인 동전이 아니다라고 할 수 없다.
물론 위 결론은 95% 정도 신뢰할 수 있다.
왜냐하면 위 구간계산을 신뢰수준인 95%에 맞추어 진행하였기 때문이다.

이번에는 앞면이 540회 나왔다고 가정하자. 그러면

* $\mu = 540/1000 = 0.54$
* $\sigma = \sqrt{\frac{p \cdot (1-p)}{1000}} = 0.0158$

그리고 신뢰수준 95%에 대한 신뢰구간은 아래와 같다.

$$[0.5091, 0.5709]$$

In [15]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

그런데, 0.5가 포함되지 않으며, 동전이 정상적이다 라는 귀무가설을 기각한다.
즉, 신뢰수준 95%로 해당 동전이 정상적이지 않다고 결론낸다.
물론 이런 결론이 틀릴 가능성도 5%이다.

### 신뢰수준과 유의수준

신뢰수준 설명에서 암시되었듯이 신뢰수준과 유의수준은 상호 보완관계이다.
95% 수준으로 신뢰할 수 있다는 말은 5% 정도 틀릴 수 있으니 유의하라는 의미와 동일한 의미임을
생각하면 바로 이해할 수 있을 것이다.

## $p$-값 해킹

$p$-값 해킹은 어떤 데이터에 대해서도 참을성 있게 잘 찾으면 원하는 가설을 입증하는 실험을 할 수 있다는 의미이다.

앞서 살펴보았듯이 가설검정은 100% 확신을 주지 못한다. 
유의수준 5%의 가설검정은 제1종 오류를 범할 수 있는 가능성이 5%라는 것을 함의한다.
비록 5%가 매우 낮은 확률이지만 0%는 아니기에 언젠가는 발생한다.

로또의 경우가 대표적이다. 1등 번호를 맞출 확률은 5%보다 훨씬 낮지만 누군가는 1등 번호를 맞춘다.
한 마디로 말해 세상일이 원래 그러며, 우주가 그렇게 돌아간다. 

아래 모의실험은 
동전을 1,000번 던져서 앞면이 나온 횟수가 469회 이하, 또는 531회 이상인지를 확인하는 실험을
1,000번 반복한다.
1,000번의 반복실험 중에 46회에서 앞면이 469회 이하, 또는 531회 이상로 발생한다.

이는 놀랍게도 5%(46/1000 = 0.046)에 가까운 수치이며,
앞서 말한대로 우주가 원래 그렇게 돌아간다는 말을 다시 한 번 확인시켜준다.

In [16]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

num_rejections

46

## A/B 테스트

A/B 테스트는 A를 선택할 확률과 B를 선택할 확률을 비교해서 두 확률 사이에 심각한 차이가 있는가를 실험하는 방법이다.

A/B 테스트는 웹 및 앱 기반 서비스에서 사용자들의 편의와 사업성을 높이기 위해 많이 사용한다. 
특히 웹 상에서 사용자들의 행동을 추적한 결과를 정량적으로 측정하여, 
사용자들의 심리와 행동을 분석하여 마케팅 전략에 활용한다.

A/B 테스트를 활용해 큰 효과를 낸 다양한 사례는 
[The Ultimate Guide To A/B Testing](https://www.smashingmagazine.com/2010/06/the-ultimate-guide-to-a-b-testing/)
에서 확인할 수 있다.

### 예제: 광고 성공률 A/B 테스트

어떤 제품을 위해 두 개의 광고 A와 B를 만들었고,
어떤 광고를 사용해야 할지 결정해야 한다.
이를 위해 제품 사이트를 방문하는 이들 중 무작위로 $N_A$ 명에게는 $A$ 광고를,
$N_B$ 명에게는 B 광고를 보여주고
광고를 클릭하는 횟수 $n_A$와 $n_B$를 확인하였다.

이제 A, B 두 광고중에 어떤 광고를 사용해야 할까?

### 광고 성공률 확률분포

이 질문에 대답하기 위해 가설 검정을 사용해보자.
그러려면 먼저 확률분포를 정해야 한다.

A 광고를 볼 때 클릭하는가를 성공이라고 하고, 그렇지 않은 경우를 실패라고 한다면,
A 광고 성공률의 분포를 사용할 수 있다.
제품 사이트 방문자중 임의로 $N_A$ 명의 표본으로만 조사했기 때문에 
진짜 성공률은 모른다.
하지만 $N_A$ 가 충분히 크면 
$p_A = \frac{n_A}{N_A}$ 가 정규분포를 따르며,
성공률의 표준편차는 $\sigma_A = \sqrt{\frac{p_A (1-p_A)}{N_A}}$ 이다.
즉, 다음이 성립한다.

$$X_A \sim N(p_A, \sigma_A^2)$$

B 광고에 대해서도 동일한 설명을 할 수 있으며 다음이 성립한다.

$$X_B \sim N(p_B, \sigma_B^2)$$

여기서 $p_B = \frac{n_B}{N_B}$, $\sigma_B = \frac{p_B (1-p_B)}{N_B}$ 이다.

**주의:** 신뢰구간을 설명할 때 사용했던 동전 던지기의 경우와 설명이 동일하다.

### 정규분포의 연산

서로 독립인 두 개의 정규분포 $X$와 $Y$가 주어졌다고 하자.

$$X \sim N(\mu_X, \sigma_X^2) \qquad \qquad Y \sim N(\mu_Y, \sigma_Y^2)$$

그러면 두 확률변수 $X$, $Y$의 합과 차도 정규분포를 따르며 다음이 성립한다. 

$$X+Y \sim N(\mu_X + \mu_Y, \sigma_X^2 + \sigma_Y^2)
\qquad\qquad
X-Y \sim N(\mu_X - \mu_Y, \sigma_X^2 + \sigma_Y^2)$$

또한 확률변수 $X$의 상수배 역시 정규분포를 따르면 다음이 성립한다.

$$cX + b \sim N(c\mu_X, c^2\sigma_X^2)$$

#### A/B 광고 성공률 차이의 확률분포

A 광고의 성공률과 B 광고의 성공률의 차이에 의미를 둘 수 있는가를 확인해야 한다.
따라서 $X_A$와 $X_B$의 차이의 확률분포인 $X_B - X_A$의 확률분포를 이용하여 가설검정을 진행해보자.

A 광고와 B 광고를 보여주는 사람들 집단을 서로 독립적으로 선택한다면
$X_A$와 $X_B$가 서로 독립이 되도록 만들 수 있다.
그러면 앞서 말한대로 $X_B - X_A$ 역시 정규분포를 따르며, 더 나아가
$Z = \frac{X_B - X_A}{\sqrt{\sigma_X^2 + \sigma_Y^2}}$
의 표준편차는 1이 된다.

**주의:** 엄격히 말하면 t-분포를 사용해야 하지만 데이터셋이 크면 정규분포를 사용해도 상관 없다.

### 가설 검정 활용 광고 성공률 차이 확인

A 광고를 본 이들 중에 200명이 광고를 클릭했고,
B 광고를 본 이들 중에 180명이 광고를 클릭했다고 하자.
이제
가설 검정을 통해 성공률 차이에 유의미가 있는지 확인해 보자.

'두 광고 성공률의 차이가 없다'를 귀무가설로 하자.

그러면 $Z = \frac{X_B - X_A}{\sqrt{\sigma_X^2 + \sigma_Y^2}}$는 표준정규분포를 따른다.

그리고 200명과 180명의 차이가 $Z$ 확률분포에서는 -1.14에 해당한다.

In [17]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic(1000, 200, 1000, 180)

print(z)

-1.1403464899034472


이 값에 대해 유의수준 5%로 양측검정 $p$-값을 계산하면 0.127이다.
이는 0.025보다 월등히 크며, 기각역 밖에 머무른다.
따라서 귀무가설을 기각할 수 없으며, 
두 광고의 성공률의 차이에 어떤 의미를 둘 수 없다.

In [18]:
# 책에서는 아래 결과에 두 배가 나와야 한다.

two_sided_p_value(z)

0.254141976542236

<img src="../images/p-value_114.png" width="50%">

이번에는 B 광고의 클릭수가 150이라고 하자.
그러면 200명과 150명의 차이가 $Z$ 확률분포에서는 -2.949에 해당한다.

In [19]:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)

-2.948839123097944


이 값에 대해 유의수준 5%로 양측검정 $p$-값을 계산하면 0.002이다.

이는 두 광고의 효과가 동일하다면 발생할 확률이 0.15%에 불과한 일이 발생했음을 말한다.
따라서 귀무가설을 기각할 수 있으며,
두 광고의 성공률이 유의미한 차이를 갖는다고 할 수 있다.

In [20]:
two_sided_p_value(z)

0.003189699706216853

<img src="../images/p-value_294.png" width="50%">

In [21]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)